# Use Mistral in Azure AI and Azure ML

Use `mistralai` client to consume Mistral deployments in Azure AI and Azure ML. Configure it as follows:

- `endpoint`: Use the endpoint URL from your deployment. Do not include either `v1/chat/completions` as this is included automatically by the client.
- `api_key`: Use your API key.

Notice that Mistral supports only chat completions API.

## Prerequisites

Install `mistralai`:

```bash
pip install mistralai
```

## Example

The following is an example about how to use `mistralai` with a Mistral model deployed in Azure AI and Azure ML:

In [ ]:
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage

In [ ]:
client = MistralClient(
    endpoint="https://<endpoint>.<region>.inference.ai.azure.com",
    api_key="<key>"
)

> Tip: Alternatively, you can configure your API key in the environment variables `MISTRAL_API_KEY`.

In [ ]:
chat_response = client.chat(
    model="azureai",
    messages=[ChatMessage(
        role="user", 
        content="Who is the most renowned French painter? Provide a short answer."
    )],
    max_tokens=50
)

The generated text can be accessed as follows:

In [ ]:
print(chat_response.choices[0].message.content)